In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

devices = tf.config.list_physical_devices()
print(devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# 디바이스를 CPU로 설정
# tf.config.experimental.set_visible_devices(devices[0], 'CPU')

In [3]:
# 디바이스를 GPU로 설정
# tf.config.experimental.set_visible_devices(devices[1], 'GPU')

- 0차 텐서: 벡터가 없기 때문에 스칼라임
- 1차 텐서: 벡터가 하나 있기 때문에 벡터임
- 2차 텐서: 벡터가 두 개 일렬로 있기 때문에 행렬이나 다이애드임
- 3차 텐서: 벡터가 세 개 일렬로 있기 때문에 트라이애드임 

In [4]:
X = tf.constant([[1, 0], [1, 2]], tf.float32)
Y = tf.constant([[2], [4]], tf.float32)

beta = tf.Variable([[0.01], [0.01]], tf.float32)

residuals = Y - tf.matmul(X, beta)

print(residuals)

Metal device set to: Apple M1
tf.Tensor(
[[1.99]
 [3.97]], shape=(2, 1), dtype=float32)


2022-06-06 22:08:24.710092: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-06 22:08:24.710210: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
s1 = tf.constant(5, tf.float32)
s2 = tf.constant(15, tf.float32)

s1s2_sum = tf.add(s1, s2)
s1s2_product = tf.multiply(s1, s2)

print(s1s2_sum)
print(s1s2_product)

tf.Tensor(20.0, shape=(), dtype=float32)
tf.Tensor(75.0, shape=(), dtype=float32)


In [6]:
s1s2_sum = s1 + s2
s1s2_product = s1 * s2

print(s1s2_sum)
print(s1s2_product)

tf.Tensor(20.0, shape=(), dtype=float32)
tf.Tensor(75.0, shape=(), dtype=float32)


In [7]:
A = tf.constant([5, 10, 7, 3, 2, 15])
B = tf.constant([5, 10, 7, 3, 2, 15])

# 요소별 곱셈
C = tf.multiply(A, B)
print(C)

C = A*B
print(C)

tf.Tensor([ 25 100  49   9   4 225], shape=(6,), dtype=int32)
tf.Tensor([ 25 100  49   9   4 225], shape=(6,), dtype=int32)


In [8]:
# 점곱: 요소별 곱셈의 총 합

c = tf.tensordot(A, B, axes = 1)
print(c.numpy())

412


In [9]:
# 행렬 곱셈

A = tf.constant([[5, 10], [3, 4]])
B = tf.constant([[1, 2, 3], [4, 5, 6]])

C = tf.matmul(A, B)
print(C.numpy())

[[45 60 75]
 [19 26 33]]


In [10]:
# 스칼라-텐서 덧셈과 곱셈 수행하기

A = tf.constant([[5, 10], [3, 4]], tf.float32)
B = tf.constant(3.0)

C = A + B
print(C.numpy())

C = A * B
print(C.numpy())

[[ 8. 13.]
 [ 6.  7.]]
[[15. 30.]
 [ 9. 12.]]


In [11]:
# 랜덤 텐서 정의하기

images = tf.random.uniform((64, 256, 256)) # 임의의 3-텐서 images를 정의함(256*256이 64개)

transform = tf.random.normal((256, 256)) # 임의의 2-텐서 transform을 정의함(256*256)

# 배치 행렬 곱셈 수행

batch_matmul = tf.matmul(images, transform) # images의 64개 256*256 행렬 각각에 transform을 곱해줌
print(batch_matmul.shape)

# 배치 요소별 곱셈을 수행

batch_elementwise = tf.multiply(images, transform)
print(batch_elementwise.shape)

(64, 256, 256)
(64, 256, 256)


## 그래디언트(gradient)
- 기울기(gradient 그레이디언트[*]) 또는 경도란 벡터 미적분학에서 스칼라장의 최대의 증가율을 나타내는 벡터장을 뜻한다.
- 기울기를 나타내는 벡터장을 화살표로 표시할 때 화살표의 방향은 증가율이 최대가 되는 방향이며, 화살표의 크기는 증가율이 최대일 때의 증가율의 크기를 나타낸다.
- 스칼라 함수 f(x)의 기울기는 ∇f로 표현한다. ∇ 기호는 벡터 미분 연산자로 나블라(nabla) 혹은 델(del)연산자라고 부른다.
- 기울기는 f의 각 성분의 편미분으로 구성된 열벡터로 정의하며 다음과 같이 표시한다.
![](./image/gradient.png)

# 자코비(jacobian)
- 벡터 미적분학에서, 야코비 행렬은 다변수 벡터 함수의 도함수 행렬이다. 야코비 행렬식(Jacobian determinant)은 야코비 행렬의 행렬식을 뜻한다.
- Jacobian(야코비언)은 어떤 다변수 벡터함수에 대한 일차 미분으로 볼 수 있습니다.
- 앞서 나온 gradient나 Jacobian이나 모두 함수에 대한 일차 미분을 나타낸다는 점에서는 동일합니다. 다만, 그레디언트는 다변수 스칼라 함수에 대한 일차 미분인 반면 Jacobian은 다변수 벡터 함수에 대한 일차미분입니다. 즉, 그레디언트는 통상적인 일변수 함수의 일차미분을 다변수 함수로 확장한 것이고, Jacobian은 이를 다시 다변수 벡터함수로 확장한 것입니다.
- Jacobian이나 그레디언트나 모두 함수에 대한 일차미분이기 때문에 미분이 가지고 있는 의미나 성질은 모두 동일하게 적용됩니다. 즉, 어떤 함수의 지역적인 변화특성을 파악할 때, 지역적인 함수의 변화를 선형근사할 때 또는 함수의 극대(극소)를 찾을 때 활용될 수 있습니다.
![](./image/jacobian.png)

## 예시)
### f1(x1, x2) = 2*x1*x2
### f2(x1, x2) = x1^2 - x2^2
### J = [[2*x2, 2*x1], [2*x1, -2*x2]]

# 헤세(hessian)
- 미적분학에서, 헤세 행렬(Hessian matrix)은 어떤 함수의 이계도함수를 행렬로 표현한 것이다. 헤세 행렬은 다변수함수가 극값을 가질 때, 그것이 극대인지, 극소인지 판정할 때 사용한다.
- 앞서 설명한 gradient, Jacobian이 모두 함수에 대한 일차미분를 나타내는 반면 Hessian은 함수의 이차미분(second derivative)를 나타낸다는 점에서 차이가 있음
- 즉, Hessian은 함수의 곡률(curvature) 특성을 나타내는 행렬로서 최적화 문제에 적용할 경우 Hessian을 이용하면 다음 식과 같이 p 근처에서 함수를 2차 항까지 근사시킬 수 있음
- 또한 Hessian은 critical point의 종류를 판별하는데 활용될 수 있습니다. 어떤 함수의 일차미분이 0이 되는 되는 점을 critical point (또는 stationary point)라 부르는데 함수의 극점(극대, 극소), saddle point(안장점: 안장점은 다변수 실함수의 변역에서, 어느 방향에서 보면 극대값이지만 다른 방향에서 보면 극소값이 되는 점이다. 수학에서 안장점이란 정류점이지만 극값을 가지지 않는 점을 말한다.) 등이 여기에 해당됨 (쉽게 생각하면, 고교수학 미분에서 f'(x) = 0이 되는 지점)
- 어떤 (다변수) 함수를 최적화시키기 위해 극점(극대, 극소)을 찾기 위해서는 먼저 그 함수의 일차미분인 gradient가 0이 되는 지점(critical point)을 찾습니다. 그런데, 이렇게 찾은 critical point가 극대점인지 극소점인지, 아니면 saddle point(말안장처럼 방향에 따라서 극대, 극소가 바뀌는 점)인지 구분하기 위해서는 이차미분값을 조사해야 하는데 이 때 바로 Hessian을 사용할 수 있습니다.
- 그 구체적인 방법은, 어떤 함수의 critical point에서 계산한 Hessian 행렬의 모든 고유값(eigenvalue)이 양수(positive)이면 해당 지점에서 함수는 극소, 모든 고유값이 음수이면 극대, 음의 고유값과 양의 고유값을 모두 가지면 saddle point인 것으로 판단합니다 (고교수학에서 f''(x)의 부호에 따라서 아래로 볼록, 위로 볼록 여부를 구분하는 것과 같은 이치).
- 그 구체적인 방법은, 어떤 함수의 critical point에서 계산한 Hessian 행렬의 모든 고유값(eigenvalue)이 양수(positive)이면 해당 지점에서 함수는 극소, 모든 고유값이 음수이면 극대, 음의 고유값과 양의 고유값을 모두 가지면 saddle point인 것으로 판단함 (고교수학에서 f''(x)의 부호에 따라서 아래로 볼록, 위로 볼록 여부를 구분하는 것과 같은 이치).
- 이러한 구분의 핵심에는 Hessian 행렬의 고유벡터(eigenvector)는 함수의 곡률이 큰 방향벡터를 나타내고 고유값(eigenvalue)은 해당 고유벡터(eigenvector) 방향으로의 함수의 곡률(curvature, 이차미분값)을 나타낸다는 점에 있음
![](./image/hessian.png)

# 출처: https://darkpgmr.tistory.com/132

## 예시)
### f(x1, x2) = (x1^2)*x2 - 2*x2^2
### Hf = [[2*x2, 2*x1], [2*x1, -4]]

# 고윳값과 고유 벡터
- 선형대수학에서, 선형 변환의 고유 벡터(eigenvector vector)는 그 선형 변환이 일어난 후에도 방향이 변하지 않는, 0이 아닌 벡터이다. 고유 벡터의 길이가 변하는 배수를 선형 변환의 그 고유 벡터에 대응하는 고윳값(eigenvalue value)이라고 한다. 선형 변환은 대개 고유 벡터와 그 고윳값만으로 완전히 설명할 수 있다.
- “벡터 x에 어떠한 선형변환 A를 했을 때, 그 크기만 변하고 원래 벡터와 평행한 벡터 x는 무엇인가요?” & “그렇다면, 그 크기는 얼마만큼 변했나요?”

# 선형 변환
- 두 벡터 공간 사이의 함수
- 예) 2차원 공간에서 [[0, -1], [1, 0]] = 시계 반대 방향 90도 회전

# 출처: https://angeloyeo.github.io/2019/07/17/eigen_vector.html, https://ko.wikipedia.org/wiki/고윳값과_고유_벡터

In [12]:
# TensorFlow에서 도함수 계산하기

x = tf.constant(2.0) # x를 상수로 정의

with tf.GradientTape() as t: # gradient tape instance 내에 f(g(x))를 정의함
    t.watch(x)
    y = x**3
    f = 5*y**2

df_dx = t.gradient(f, x)
print(df_dx.numpy())

# with절: 자원을 획득하고, 사용하고, 반납할때 주로 사용

# f = open('myFile.txt', 'w', encoding='utf8')
# f.write("test")
# f.close()

# 위 코드를 아래와 같이 with절로 표현할 수 있음

# with open('mytextfile.txt', 'r', encoding='utf8') as f:
#     f.wirte("test")

960.0


# 자동 미분
#### 자동미분은 컴퓨터 프로그램에 의해서 구체화된 함수의 미분을 산술적으로 계산할 때 사용하는 기술의 집합을 말합니다.

#### 컴퓨터 프로그램에서 구체화한 함수는 아무리 복잡해보이더라도 기본적인 산술 연산 (덧셈, 뺄셈, 곱셉, 나눗셈 등)과 기본적인 함수 (지수, 로그, 싸인, 코싸인 등) 의 연속적인 실행으로 이루어진다는 아이디어를 기반으로 합니다.

#### 복잡한 함수도 연쇄 법칙(Chain Rule)을 이용함으로써 합성함수를 구성하는 각 기본함수의 미분의 곱으로 나타내고 반복적으로 계산함으로써 자동으로 복잡한 함수를 정확하고 효율적으로 미분할 수 있습니다.


# 출처: https://rfriend.tistory.com/556 [R, Python 분석과 프로그래밍의 친구 (by R Friend):티스토리]

# TensorFlow에서 사용할 데이터 로딩

In [13]:
# TensorFlow 2는 기본적으로 즉시 실행을 사용하므로 동적 계산 그래프 사용
# 동적 계산 그래프(Define-by-Run)
# '데이터 흘려보내기'와 '계산 그래프 구축'이 동시에 이루어짐
# 계산 그래프를 컴파일하여 변환할 필요가 없음(특정 도메인 특화 언어를 사용하지 않고 if, for문 등을 그대로 사용 가능, 파이썬 프로그램 형태로 실행됨)

In [14]:
# numpy로 이미지 데이터 임포트하기
images = np.random.uniform(0, 256, size=(32, 64, 64, 3))
images = np.save('./image/images.npy', images) # 이미지 데이터 만들기

In [15]:
images = np.load('./image/images.npy') # numpy를 사용해 이미지 데이터를 가져오기

images = images / 255.0 # 픽셀 값을 [0, 1] 간격으로 정규화

print(images.shape)

(32, 64, 64, 3)


In [16]:
plt.imsave('./image/images.jpeg', images) # 위 images.npy 파일을 images.jpeg로 저장

In [17]:
# 상수 텐서를 사용해 tensorflow에서 나누기 수행하기

images = np.load('./image/images.npy') # numpy를 사용해 이미지 데이터를 가져오기

images = tf.constant(images) # numpy 배열을 tensorflow 상수로 변환

images = images / 255.0 # 픽셀 값을 [0, 1] 간격으로 정규화

In [18]:
# division 메서드를 사용해 tensorflow에서 나누기 수행하기

images = np.load('./image/images.npy') # numpy를 사용해 이미지 데이터를 가져오기

images = tf.divide(images, 255.0)

In [19]:
# TensorFlow에서 사용하기 위해 데이터를 로드할 때 기억해야 할 두 가지 사항
# 1. numpy와 pandas를 포함해 데이터를 로드하려는 모든 모듈을 사용할 수 있음. tensorflow는 데이터 임포트 기능도 제공
# 2. 데이터를 가져온 다음 tensorflow 연산에 사용하기 전에 상수나 변수와 같은 numpy 배열이나 tensorflow 객체로 변환해야 함

In [29]:
# sample csv 파일 만들기
data = [[1,2,3,4], [5,6,7,8]]

dataframe = pd.DataFrame(data)
dataframe.to_csv('./image/sample_csv.csv', index=False)

In [30]:
data = pd.read_csv('./image/sample_csv.csv') # pandas를 사용해 데이터 가져오기
print(data)

data_tensorflow = tf.constant(data) # 데이터를 tensorflow 상수로 변환

data_numpy = np.array(data) # 데이터를 numpy 배열로 변환

   0  1  2  3
0  1  2  3  4
1  5  6  7  8


In [31]:
print(data_tensorflow)

tf.Tensor(
[[1 2 3 4]
 [5 6 7 8]], shape=(2, 4), dtype=int64)


In [32]:
print(data_numpy)

[[1 2 3 4]
 [5 6 7 8]]
